In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import os
import joblib
import dask.dataframe as dd

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../data'
# datadir = '../../../data'

mode = 'ON_DEMAND_AUTO'
simulation_id = 'dev'
test = False

#Created in the EIA_data_download.ipynb notebook
#existing_load=pd.read_csv('../../data/EIA_demand_summary.csv') 
existing_load=pd.read_csv(os.path.join(datadir,'EIA_demand_summary.csv'))
if test:
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))
    # df = df.loc[df['mode'] == mode]
    # df = df.sample(frac=0.001, random_state=1)
    # df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
    simulation_id = 'dev'

else: 


    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]

### get only the tnc trips

In [2]:
df = df.loc[df['mode'] == 'ON_DEMAND_AUTO']
df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
df = df.compute()

Two ways to do this: 
 1. everyone charges when they are ~empty only. in this case they would take as many trips as they need and charge at either a L2 or DCFC charger for as long as they need to get back to full. 
 Obviously not realistic. 

 2. Charge after x trips and then charge as long as needed to get back the energy lost in those x trips as long as dont exceed some threshold distance then stop and charge. Charging is drawn from a distribution of L2 and DCFC. 

In [9]:
df

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type
npartitions=566,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,timedelta64[ns],timedelta64[ns],float64,object,object,float64,float64,object,float64,float64,object,object,object,object,object,object,object,object,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,int64,object,object,object,object,float64,object,object,object,object,object,object,object,object,object,object,object,object,float64,object,object,object,object,object,float64,float64,float64,float64,float64,float64,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
max_trip_distance = 200
max_trips = 9
tot_distance = 0
trip_count = 0
kwh_per_mi = 0.3
which_charge_rate = sim.determine_charger_availability_tnc()
charge = 0

for row in range(len(df)):
    trip = df.iloc[row,]
    tot_distance += trip['distance_miles']
    if tot_distance > max_trip_distance:
        # exceeded the distance limit
        # start a new trip
        trip_count = 0
        tot_distance = 0
        
    if trip_count > max_trips:
        # we should charge
        charge += tot_distance * kwh_per_mi # mi * kwh/mi 
        # add to load here
        # calc time to load up to max distance
        #  determine_charger_availability_tnc in kw/h
    
    # charge duration = charge_used / charge rate
    charge_duration = charge * which_charge_rate

    # this needs to get saved somewere. 
    charge_time = trip['end_time'] + pd.Timedelta(f"{charge_duration} hour")

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# this is the instance where we see how many trips it took to get to 
tot_distance = 0
for row in len(df)
    trip = df.iloc[row,]
    tot_distance += trip['distance_miles']
    if tot_distance > 100:
        tot_distance = 0

# What about a fraction of the private_auto? 

matching the tnc vmt with the source from this: https://www.fehrandpeers.com/what-are-tncs-share-of-vmt/

they find %2 of vmt is due to uber and lyft

In [2]:
df_full = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))
df_thu = df_full.loc[(df_full['mode'] == 'PRIVATE_AUTO') & (df_full['weekday'] == 'thursday')]

In [4]:
df_thu['person_id'].value_counts()

person_id
10362711284651667896    54
6681715039027230810     48
12850282687603040007    45
16239242443702864114    44
5367467126005835109     44
                        ..
8617319745911718296      1
3902265398452998342      1
3902258746108093852      1
16372705720597263146     1
17753246965725389032     1
Name: count, Length: 4652231, dtype: int64

In [21]:
def calc_tnc_vmt(df, trip_count=12, return_df=False):
    """calculate the vmt for tncs

    Args:
        df (_type_): _description_

    Returns:
        _type_: _description_
    """
    col = 'person_id'
    # calculate how many trips each person took
    counts = df[col].value_counts()

    # only count those that took > trip_count trips
    valids = counts[counts>trip_count].index

    ubers = df[df[col].isin(valids)]
    # vmt per driver 
    # vmt = ubers.groupby(col)['distance_miles'].sum()

    vmt_tot = ubers['distance_miles'].sum()

    if return_df:
        return ubers
    return vmt_tot

In [3]:
df_thu = df_full.loc[(df_full['mode'] == 'PRIVATE_AUTO') & (df_full['weekday'] == 'thursday')]

In [12]:
df_thu_all_modes = df_full.loc[(df_full['weekday'] == 'thursday')]
df_sat_all_modes = df_full.loc[(df_full['weekday'] == 'saturday')]

In [14]:
vmt_thu = calc_tnc_vmt(df_thu, trip_count=10)
# total vmt should be ~2% of total vmt
vmt_thu / df_thu_all_modes['distance_miles'].sum()

0.019336600737262198

In [13]:
df_sat = df_full.loc[(df_full['mode'] == 'PRIVATE_AUTO') & (df_full['weekday'] == 'saturday')]

0.003932185889004563

In [20]:
vmt_sat = calc_tnc_vmt(df_sat, trip_count=9)
vmt_sat / df_sat_all_modes['distance_miles'].sum()

0.014490599428222412

Lets get those drivers that have the trips above the threshold to match the percentage of VMT

In [22]:
ubers_sat = calc_tnc_vmt(df_sat, trip_count=9, return_df=True)
ubers_thu = calc_tnc_vmt(df_thu, trip_count=10, return_df=True)

In [24]:
ubers = pd.concat([ubers_sat, ubers_thu])

Save this to avoid calculating it again

In [26]:
ubers.to_parquet(os.path.join(datadir, 'tnc_trips.parquet'))